In [111]:
import pandas as pd
import featuretools as ft
import numpy as np
import os


Index = {
    (0 , 0): 0,
    (-1, 0): 1,
    (1 , 0): 2,
    (0 ,-1): 3,
    (0 , 1): 4
}

raw = pd.read_json("C:/Users/vikto/Desktop/Pacman-Pipeline/data/raw/sessions/session_12.json")

df = pd.json_normalize(raw["ticks"])

df.dropna
df.columns


Index(['enemy_momenta', 'enemy_positions_grid', 'enemy_positions_screen',
       'player_buffer', 'player_momentum', 'player_position_grid',
       'player_position_screen', 'score', 'tick'],
      dtype='object')

In [112]:
#read the tick_id and score into a seperate tick table

ticks = df["tick"].values
score = df["score"].values

tick_data = {
    "tick_id" : ticks,
    "score" : score
}

tick_table = pd.DataFrame(data=tick_data)
tick_table.head()


,tick_id,score
0,0,0
1,10,0
2,20,0
3,30,0
4,40,0


In [ ]:
#read the player specific data into a seperate player table

posX = df["player_position_screen"].str[0]
posY = df["player_position_screen"].str[1]


#shift the player momentum buffer by -1 to get the players reaction to the game-state
df["player_buffer"] = df["player_buffer"].shift(-1)

df.drop(df.tail(1).index, inplace=True)

#convert the player momentum buffer into the corresponding bufferIndex
bufferIndex = df["player_buffer"].apply(lambda x: Index[tuple(x)])

#convert the player momentum into the corresponding momentumIndex
momentumIndex = df["player_momentum"].apply(lambda x: Index[tuple(x)])

player_data = {
    "tick" : ticks, 
    "posX" : posX,
    "posY" : posY, 
    "player_bufferIndex" : bufferIndex, 
    "momentum_Index" : momentumIndex
}

player_table = pd.DataFrame(data=player_data) 

#drop the last row because there is no momentum buffer for it
player_table.drop(player_table.tail(1).index, inplace=True)

player_table.tail()

,tick_id,posX,posY,player_bufferIndex,momentum_Index
3231,20570,714.0,440.0,2.0,1.0
3232,20580,694.0,440.0,0.0,1.0
3233,20590,688.0,440.0,4.0,2.0
3234,20597,702.0,440.0,4.0,2.0
3235,20600,708.0,440.0,4.0,2.0


In [114]:
#read the enemy specific data into a seperate enemy table

#unnest the momenta and screen position arrays
df_long = df.explode(["enemy_momenta", "enemy_positions_screen"]).reset_index(drop=True)

df_long["enemy_id"] = df_long.groupby("tick").cumcount()


df_long[["posX", "posY"]] = pd.DataFrame(df_long["enemy_positions_screen"].tolist(), index=df_long.index)


df_long = df_long.drop(columns=["enemy_positions_screen"])

enemy_id = df_long["enemy_id"].values
momentumIndex = df_long["player_momentum"].apply(lambda x: Index[tuple(x)])
posX = df_long["posX"]
posY = df_long["posY"]

ticks = df_long["tick"].values

enemy_data = {
    "tick" : ticks, 
    "enemy_id" : enemy_id,
    "momentumIndex" : momentumIndex,
    "posX" : posX,
    "posY" : posY
}

enemy_table = pd.DataFrame(data=enemy_data)

enemy_table.head()

,tick,enemy_id,momentumIndex,posX,posY
0,0,0,0,840.0,120.0
1,0,1,0,120.0,760.0
2,0,2,0,1160.0,760.0
3,10,0,0,820.0,120.0
4,10,1,0,120.0,740.0


In [ ]:
es = ft.EntitySet(id="game_data")
#add the tick table to the entity set
es = es.add_dataframe (
    dataframe=tick_table,
    dataframe_name="ticks",
    index="tick_id",
    make_index=False,
    already_sorted=True
)

#add the player table to the entity set
es = es.add_dataframe (
    dataframe=player_table,
    dataframe_name = "player",
    index="tick_id",
    time_index="tick"
    make_index=True,
    already_sorted=True
)

#add the enemy table to the entity set
es = es.add_dataframe (
    dataframe=enemy_table,
    dataframe_name="enemy",
    index = "tick_id",
    time_index="tick"
    make_index = True,
    already_sorted= True
)

#add the relationships between the tables
es.add_relationship(
    parent_dataframe_name="ticks",
    parent_column_name="tick_id",
    child_dataframe_name="player",
    child_column_name="tick_id"
)

es.add_relationship(
    parent_dataframe_name="ticks",
    parent_column_name="tick_id",
    child_dataframe_name="enemy",
    child_column_name="tick"
)



ValueError: Unable to add relationship because child column 'tick_id' in 'player' is also its index